In [ ]:
%%capture
import os
import pandas as pd
import numpy as np
from dj_notebook import activate
from pathlib import Path

env_file = os.environ["INTECOMM_ENV"]
analysis_folder = Path(os.environ["INTECOMM_ANALYSIS_FOLDER"])
reports_folder = Path(os.environ["INTECOMM_ANALYSIS_FOLDER"])
plus = activate(dotenv_file=env_file)


In [ ]:
from intecomm_analytics.dataframes import get_df_main_1858
from intecomm_analytics.dataframes import treatment_arm_labels as treatment_arm
from tabulate import tabulate
from edc_constants.constants import NO, YES
from intecomm_rando.constants import FACILITY_ARM, COMMUNITY_ARM

In [ ]:
df_main = get_df_main_1858(analysis_folder)

In [ ]:
df_main[(df_main.endline_datetime - df_main.baseline_datetime).dt.days < 182].offstudy_reason.value_counts()

In [ ]:
# this is the htn only and htn+dm --- excludes HIV and DM only
# htn_and_dm_cond = (((df_all.htn==1) & (df_all.dm==1)) | ((df_all.htn==1) & (df_all.dm==0))) & (df_all.hiv==0)

In [ ]:
def get_yes_no_na(value):
    if value == 1:
        return YES
    elif value == 0:
        return NO
    return np.nan


df_main["bp_controlled_baseline_str"] = df_main["bp_controlled_baseline"].apply(get_yes_no_na)
df_main["bp_controlled_endline_str"] = df_main["bp_controlled_endline"].apply(get_yes_no_na)
df_main["bp_severe_htn_baseline_str"] = df_main["bp_severe_htn_baseline"].apply(get_yes_no_na)
df_main["bp_severe_htn_endline_str"] = df_main["bp_severe_htn_endline"].apply(get_yes_no_na)

columns = ["subject_identifier", "bp_sys_baseline", "bp_dia_baseline","bp_sys_endline", "bp_dia_endline", "bp_controlled_baseline", "bp_controlled_baseline_str", "bp_controlled_endline", "bp_controlled_endline_str", "bp_severe_htn_baseline", "bp_severe_htn_baseline_str", "bp_severe_htn_endline","bp_severe_htn_endline_str", "ncd", "hiv", "dm", "htn", "assignment"]

In [ ]:
df_all = df_main[(((df_main.htn==1) & (df_main.dm==1)) | ((df_main.htn==1) & (df_main.dm==0))) & (df_main.hiv==0)][columns].copy()
df_all.reset_index(drop=True, inplace=True)

In [ ]:
df_htn_dm = df_main[((df_main.htn==1) & (df_main.dm==1)) & (df_main.hiv==0)][columns].copy()
df_htn_dm.reset_index(drop=True, inplace=True)
df_htn_dm

In [ ]:
df_htn_only = df_main[(df_main.htn==1) & (df_main.dm==0) & (df_main.hiv==0)][columns].copy()
df_htn_only.reset_index(drop=True, inplace=True)
df_htn_only

In [ ]:
pd.crosstab(df_all['dm'], df_all["htn"])

In [ ]:



def get_cells_for_continuous_var(df)->list[str]:
    """ From describe(), format 3 cells as:

        +======================+
        | 930                  |
        +----------------------+
        | 127.69(16.84)        |
        +----------------------+
        | 127.00(82.00–183.00) |
        +----------------------+
    """
    return [
        f"{int(df['count'])}",
        f"{df['mean']:.2f}({df['std']:.2f})",
        f"{df['50%']:.2f}({df['min']:.2f}–{df['max']:.2f})"
    ]

def get_cells_for_yes_no_na(df:pd.DataFrame, col:str, arm:str|None=None)->list[str]:
    if arm:
        n = len(df[(df['assignment']==arm) & (df[col].notna())])
        counts = df[(df['assignment'] == arm) & (df[col].notna())][col].value_counts()
        percentages = df[(df['assignment'] == arm) & (df[col].notna())][col].value_counts(normalize=True) * 100
    else:
        n = len(df[(df[col].notna())])
        counts = df[(df[col].notna())][col].value_counts()
        percentages = df[(df[col].notna())][col].value_counts(normalize=True) * 100
    return [
        n,
        f"{counts.get(NO, 0)} ({percentages.get(NO, 0):.1f}%)",
        f"{counts.get(YES, 0)} ({percentages.get(YES, 0):.1f}%)"]
        # f"{counts.get(NOT_APPLICABLE, 0)} ({percentages.get(NOT_APPLICABLE, 0):.1f}%)"]


def get_formatted_rows_sys_dia(df, label):
    """Returns 5 columns"""
    df_base = df[df[f'bp_{label}_baseline'].notna()].copy()
    df_end = df[df[f'bp_{label}_endline'].notna()].copy()

    baseline_a = df_base[df_base['assignment'] == COMMUNITY_ARM][f'bp_{label}_baseline'].describe()
    endline_a = df_end[df_end['assignment'] == COMMUNITY_ARM][f'bp_{label}_endline'].describe()

    baseline_b = df_base[df_base['assignment'] == FACILITY_ARM][f'bp_{label}_baseline'].describe()
    endline_b = df_end[df_end['assignment'] == FACILITY_ARM][f'bp_{label}_endline'].describe()

    baseline_all = df_base[f'bp_{label}_baseline'].describe()
    endline_all = df_end[f'bp_{label}_endline'].describe()

    return  {
        'Timepoint': ['Baseline', '', '', 'Endline', '', ''],
        'Statistics': ['n', 'Mean(sd)', 'Median(min-max)','n', 'Mean(sd)', 'Median(min-max)'],
        treatment_arm[COMMUNITY_ARM]: [
            *get_cells_for_continuous_var(baseline_a),
            *get_cells_for_continuous_var(endline_a),
        ],
        treatment_arm[FACILITY_ARM]: [
            *get_cells_for_continuous_var(baseline_b),
            *get_cells_for_continuous_var(endline_b),
        ],
        'All': [
            *get_cells_for_continuous_var(baseline_all),
            *get_cells_for_continuous_var(endline_all),
        ],
    }

def get_formatted_rows_controllled(df:pd.DataFrame, baseline_col:str, endline_col:str):
    """Returns 5 columns"""
    return  {
        'Timepoint': ['Baseline', '', '', 'Endline', '', ''],
        'Statistics': ['n', 'No', 'Yes', 'n', 'No', 'Yes'],
        treatment_arm[COMMUNITY_ARM]: [
            *get_cells_for_yes_no_na(df, baseline_col, arm=COMMUNITY_ARM),
            *get_cells_for_yes_no_na(df, endline_col, arm=COMMUNITY_ARM),
        ],
        treatment_arm[FACILITY_ARM]: [
            *get_cells_for_yes_no_na(df, baseline_col, arm=FACILITY_ARM),
            *get_cells_for_yes_no_na(df, endline_col, arm=FACILITY_ARM),
        ],
        'All': [
            *get_cells_for_yes_no_na(df, baseline_col),
            *get_cells_for_yes_no_na(df, endline_col),
        ],
    }


In [ ]:
# build section on systolic and diastolic at baseline and endline
# a. all HTN only and HTN+DM subjects
sys_table = {'Condition': ['All', '', '', '', '', '']}
sys_table.update({
    'Parameter': ['Blood pressure: systolic (mmHg)', '', '', '', '', ''],
    **get_formatted_rows_sys_dia(df_all, "sys")
})
sys_df = pd.DataFrame(sys_table)

dia_table = {'Condition': ['All', '', '', '', '', '']}
dia_table.update({
    'Parameter': ['Blood pressure: diastolic (mmHg)', '', '', '', '', ''],
    **get_formatted_rows_sys_dia(df_all, "dia")
})
dia_df = pd.DataFrame(dia_table)

# b. htn+dm subjects

sys_table = {'Condition': ['HTN and DM', '', '', '', '', '']}
sys_table.update({
    'Parameter': ['Blood pressure: systolic (mmHg)', '', '', '', '', ''],
    **get_formatted_rows_sys_dia(df_htn_dm, "sys"),
})
sys_htn_dm_df = pd.DataFrame(sys_table)

dia_table = {'Condition': ['HTN and DM', '', '', '', '', '']}
dia_table.update({
    'Parameter': ['Blood pressure: diastolic (mmHg)', '', '', '', '', ''],
    **get_formatted_rows_sys_dia(df_htn_dm, "dia")
})
dia_htn_dm_df = pd.DataFrame(dia_table)

# c. htn only subjects

sys_table = {'Condition': ['HTN only', '', '', '', '', '']}
sys_table.update({
    'Parameter': ['Blood pressure: systolic (mmHg)', '', '', '', '', ''],
    **get_formatted_rows_sys_dia(df_htn_only, "sys"),
})
sys_htn_only_df = pd.DataFrame(sys_table)

dia_table = {'Condition': ['HTN only', '', '', '', '', '']}
dia_table.update({
    'Parameter': ['Blood pressure: diastolic (mmHg)', '', '', '', '', ''],
    **get_formatted_rows_sys_dia(df_htn_only, "dia")
})
dia_htn_only_df = pd.DataFrame(dia_table)


In [ ]:
# if you want, you can print / export the sys dia section separately
# concat results
summary_df = pd.concat([sys_df, dia_df, sys_htn_dm_df, dia_htn_dm_df, sys_htn_only_df, dia_htn_only_df], ignore_index=True)

# generate table
summary_tab = tabulate(summary_df, headers='keys', tablefmt='grid')

# export as csv
path = reports_folder / 'bp_summary_sys_dia.csv'
summary_df.to_csv(path_or_buf=path, index=False)

# Write the table to file
path = reports_folder / 'bp_summary_sys_dia.txt'
with open(path, 'w') as file:
    file.write(summary_tab)


In [ ]:
# build the table section on bp control
# table as three subsections: ALL, HTN and DM, HTN only
# each subsection shows stats for <140/90 and >=180/120

# convert binary 0/1 values to YES/NO for printing
df_all["bp_controlled_baseline_str"] = df_all["bp_controlled_baseline"].apply(get_yes_no_na)
df_all["bp_controlled_endline_str"] = df_all["bp_controlled_endline"].apply(get_yes_no_na)
df_all["bp_severe_htn_baseline_str"] = df_all["bp_severe_htn_baseline"].apply(get_yes_no_na)
df_all["bp_severe_htn_endline_str"] = df_all["bp_severe_htn_endline"].apply(get_yes_no_na)

# a. All controlled / severe
all_controlled_table = {'Condition': ['All', '', '', '', '', '']}
all_controlled_table.update({
    'Parameter': ['Participants with blood pressure <140/90 mm Hg', '', '', '', '', ''],
    **get_formatted_rows_controllled(df_all, "bp_controlled_baseline_str", "bp_controlled_endline_str")
})

all_severe_htn_table = {'Condition': ['All', '', '', '', '', '']}
all_severe_htn_table.update({
    'Parameter': ['Participants with blood pressure >=180/120 mm Hg', '', '', '', '', ''],
    **get_formatted_rows_controllled(df_all, "bp_severe_htn_baseline_str", "bp_severe_htn_endline_str")
})

all_controlled_df = pd.DataFrame(all_controlled_table)
all_severe_htn_df = pd.DataFrame(all_severe_htn_table)

# b. htn only and htn+dm controlled / severe
htn_dm_controlled_table = {'Condition': ['HTN and DM', '', '', '', '', '']}
htn_dm_controlled_table.update({
    'Parameter': ['Participants with blood pressure <140/90 mm Hg', '', '', '', '', ''],
    **get_formatted_rows_controllled(df_htn_dm, "bp_controlled_baseline_str", "bp_controlled_endline_str")
})

htn_dm_severe_htn_table = {'Condition': ['HTN and DM', '', '', '', '', '']}
htn_dm_severe_htn_table.update({
    'Parameter': ['Participants with blood pressure >=180/120 mm Hg', '', '', '', '', ''],
    **get_formatted_rows_controllled(df_htn_dm, "bp_severe_htn_baseline_str", "bp_severe_htn_endline_str")
})

htn_dm_controlled_df = pd.DataFrame(htn_dm_controlled_table)
htn_dm_severe_htn_df = pd.DataFrame(htn_dm_severe_htn_table)

# c. htn only controlled / severe
htn_only_controlled_table = {'Condition': ['HTN only', '', '', '', '', '']}
htn_only_controlled_table.update({
    'Parameter': ['Participants with blood pressure <140/90 mm Hg', '', '', '', '', ''],
    **get_formatted_rows_controllled(df_htn_only, "bp_controlled_baseline_str", "bp_controlled_endline_str")
})

htn_only_severe_htn_table = {'Condition': ['HTN only', '', '', '', '', '']}
htn_only_severe_htn_table.update({
    'Parameter': ['Participants with blood pressure >=180/120 mm Hg', '', '', '', '', ''],
    **get_formatted_rows_controllled(df_htn_only, "bp_severe_htn_baseline_str", "bp_severe_htn_endline_str")
})

htn_only_controlled_df = pd.DataFrame(htn_only_controlled_table)
htn_only_severe_htn_df = pd.DataFrame(htn_only_severe_htn_table)



In [ ]:
# build final table df using all table dfs
final_table_df = pd.concat([
    sys_df,
    dia_df,
    all_controlled_df,
    all_severe_htn_df,
    sys_htn_dm_df,
    dia_htn_dm_df,
    htn_dm_controlled_df,
    htn_dm_severe_htn_df,
    sys_htn_only_df,
    dia_htn_only_df,
    htn_only_controlled_df,
    htn_only_severe_htn_df
], ignore_index=True)
final_table = tabulate(final_table_df, headers="keys", tablefmt="grid")

# Write final_table to text
path = reports_folder / 'bp_table_final.txt'
with open(path, 'w') as file:
    file.write(final_table)

# Write final_table_df to csv
path = reports_folder / 'bp_table_final.csv'
final_table_df.to_csv(path_or_buf=path, index=False)


In [ ]:
df_baseline = df_htn_dm[["subject_identifier", "assignment","bp_controlled_baseline"]].copy()
df_baseline.reset_index(drop=True, inplace=True)
df_baseline.rename(columns={"bp_controlled_baseline": "bp_controlled"}, inplace=True)
df_baseline["bp_controlled"] = df_baseline["bp_controlled"] == 1.0
df_baseline["time"] = "baseline"

df_endline = df_htn_dm[["subject_identifier", "assignment","bp_controlled_endline"]].copy()
df_endline.reset_index(drop=True, inplace=True)
df_endline.rename(columns={"bp_controlled_endline": "bp_controlled"}, inplace=True)
df_endline["time"] = "endline"
df_endline["bp_controlled"] = df_endline["bp_controlled"] == 1.0

df_bp = pd.concat([df_baseline, df_endline], ignore_index=True)


In [ ]:
path = analysis_folder / 'df_bp.csv'
df_bp.to_csv(path, index=False)